# Metaprogramação em Julia
Esse tutorial é baseado no Workshop em Metaprogramming de 2021 da JuliaCon ministrado por 
David P. Sanders. O material do workshop pode ser encontrado [neste repositório](https://github.com/dpsanders/Metaprogramming_JuliaCon_2021).


In [1]:
using Pkg
Pkg.activate(".")
using StatsBase: sample
using Random

  Activating project at `~/MEGA/EMAp/Julia_Tutorials/MetaProgramming`
┌ Info: Precompiling StatsBase [2913bbd2-ae8a-5f71-8c99-4fb6c76f3a91]
└ @ Base loading.jl:1423


## Introdução

Metaprogramação consiste em utilizar a própria linguagem de programação para escrever e manipular código dela mesma, e ainda
avaliar este código. Metaprogramação não é algo presente em toda linguagem. Felizmente, em Julia temos essa possibilidade,
o que nos permite gerar código de maneira bastante sofisticada.

### Exemplo Inicial

Vamos começar com um exemplo bastante simples. Suponha que você tem uma lista de variáveis que recebem um valor aleatório.
Você então que descobrir qual dessas variáveis contém um certo valor. No código abaixo, nós amostramos sem reposição
os valores 1 até 5, e colocamos dentro das variáveis `a`, `b`, `c`, `d` e `e`. Asssim, de antemão, nós não sabemos
qual valor está contido em cada uma delas. Como podemos descobrir, por exemplo, qual variável contém o valor 1?

In [2]:
Random.seed!(7)
a,b,c,x,y = sample(1:5, 5, replace=false);

Para descobrir quem tem o valor 1, podemos ir printando o valor de cada uma junto com um `if`. Porém, percebemos
que teríamos que realizar um tipo de trabalho "manual". Por exemplo:

In [3]:
for (i,v) in enumerate([a,b,c,x,y])
    v == 1 ? println(i) : nothing
end

4


O código mostrou que a quarta variável é que contém o valor 1. Agora olhamos na nossa lista `[a,b,c,x,y]` e vemos que a quarta variável é `x`. Logo ela contém o valor 1. Veja que o trabalho "manual" foi que nós tivemos que olhar qual era a variável na lista. E se quisermos evitar esse trabalho "manual" de olhar na lista nós mesmos? Como "printar" o nome na variável na tela?
Bem, sem metaprogramção, uma solução seria fazer isso na "força bruta":

In [4]:
a == 1 ? print("a") : nothing
b == 1 ? print("b") : nothing
c == 1 ? print("c") : nothing
x == 1 ? print("x") : nothing
y == 1 ? print("y") : nothing

x

Conseguimos, mas tivemos que escrever um monte de linhas de código. Para cinco variáveis isso não é problema, mas se tivéssemos 
bem mais variáveis, isso se tornaria tedioso.
É nesse tipo de problema (e coisa bem mais complexas), que a metaprogramação nos ajuda. Vamos começar
usando o comando `names(Main)`. A função `name()` recebe o nome de um `module`, e rotorna todos os nomes sendo exportados por aquele
`module`. No caso, `Main` é o `module` padrão que estamos utilizando. O `Main`, por padrão, exporta `Base`, `Core` e `Main`, e além disso,
´ele exporta toda variável criada nele. Assim, ao usar `names(Main)`, vamos pegar todos os resultados a partir do quarto índice.

In [5]:
names(Main)

8-element Vector{Symbol}:
 :Base
 :Core
 :Main
 :a
 :b
 :c
 :x
 :y

In [5]:
names(Main)[4:end]

5-element Vector{Symbol}:
 :a
 :b
 :c
 :x
 :y

Muito bem, conseguimos de maneira rápida o nome de nossas variáveis. Mas como fazemos para avaliar o valor em cada uma delas?
Novamente, sem metaprogramação, o `names(Main)` não será muito útil, porém, com metaprogramação, é bastante. Abaixo iremos apresentar
a solução, e na seção seguinte iremos explicar melhor o que foi feito e os básicos da metaprogramação em Julia.

In [6]:
for v in names(Main)[4:end]
    eval(v) == 1 ? println(v) : nothing
end

x


## Básicos da Metaprogramação

Vimos que o código anterior foi capaz de facilmente retornar a variável com o valor 1. Além disso, é facíl ver que ele funciona imediatamente
para casos envolvendo mais variáveis, sem precisar ser modificado. O que não é verdade para as outras duas soluções que apresentamos.
A nossa solução envolveu o uso de uma função chamada `eval`, que é uma função do módulo `Core`, ou seja, `eval` é o mesmo que
`Core.eval`.
Como você já deve ter adivinhado, a função `eval` recebe uma "expressão" em Julia, e avalia essa "expressão". Mas, afinal,
o que é uma "expressão"? Antes de responder a essa pergunta, precisamos introduzir o conceito de um símbolo.

### Representando Variáveis com Símbolos

Para que uma linguagem de programação possa realizar metaprogramação, é necessário que de alguma forma ela seja capaz
de distinguir quando seu código está sendo executado versus quando seu código está sendo referenciado, ou seja,
ela deve ser capaz de representar seu próprio código como dado. Por exemplo, se eu escrever `x = 1`, é preciso
representar que uma variável chamada "x" contém o valor 1... Curiosamente, acabamos de exemplificar essa situação!
Quando estamos escrevendo em português, utilizamos aspas como uma maneira de se referir a palavra em si, invés de
referenciar o que ela significa. **Isso é exatamente a idéia de um símbolo!**

A linguagem Julia precisa ser capaz
de usar aspas, ou seja, hora falar do valor por meio da variável, hora falar da própria variável.
Vejamos outro exemplo envolvendo a língua portuguesa.

*A palavra "bonita" é bonita.*

Na frase acima, a mesma palavra aparece com sentidos diferentes. Primeiro ela aparece se referindo a própria palavra, e 
depois aparece indicado o seu siginificado. Em Julia, as aspas podem ser trocadas por `:`, e agora temos uma situação parecida.
`:x` se refere à variável em si, enquanto que `x` é o valor guardado dentro da variável;

In [6]:
x = 1
typeof(:x), typeof(x)

(Symbol, Int64)

Quando escrevemos `x + 1`, o que estamos dizendo é que queremos somar o valor que está em `x` com 1, o que no nosso caso
irá retornar o valor 2. Já a expressão `:x + 1`, não faz sentido. Porém, novamente a função `eval` irá nos ajudar.

In [52]:
eval(:x) + 1 == x + 1

true

No código acima, `eval(:x)` está pedindo para que avaliemos o valor que está dentro da variável de nome "x", que é o valor 1. Ou seja,
`eval(:x)` é o mesmo que `x`.

### Construíndo e Avaliando Expressões

Podemos pensar que todo código em Julia se resume a uma combinação de símbolos e valores literais. 
Essa combinação de símbolos e valores literais é chamada de expressão.

Na prática, uma expressão é um struct do tipo 
do tipo `Expr`, conténdo um campo `head` e um campo `args`. O campo `head` sempre contém um símbolo (`Symbol`),
enquanto que `args` é um array contendo símbolos, valores literais ou outras expressões ("it's all symbols and values all the way down").
O `head` determina o tipo da expressão, enquanto os demais argumentos representam o que de fato a expressão faz.

Vamos ver um exemplo:


In [15]:
1 + 2

3

A linha de código acima é interpretada em Julia como uma expressão com os argumentos `1`, `2` e `:+`, onde os dois primeiros são valores
literais e o último é um símbolo. Essa mesma expressão pode ser representada por `Expr(:call, :+, 1, 2)`. Ou seja, é sua `head` contém
o símbolo `:call`, representando justamente que essa expressão está sendo avaliada.

In [11]:
ex = Expr(:call, :+, 1, 2)

ex.head, ex.args

(:call, Any[:+, 1, 2])

Assim, uma vez que temos uma expressão, podemos passar utilizar a função `eval`, que ela irá
executar o que está descrito na expressão.

Como vimos no exemplo acima, a estrutura de uma expressão costuma ser mais complexa do que a forma "natural"
com a qual escrevemos código. Porém, da mesma forma que um símbolo é representado usando `:`, podemos
utilizar o mesmo para expressões, e.g. `:(1 + 2 / 2)`.
Mais ainda, usando a função `Meta.parse` podemos transformar strings em expressões.

In [16]:
ex_string = "1 + 2 / 2"
ex = Meta.parse(ex_string)

:(1 + 2 / 2)

Caso queiramos entender melhor como está estruturada uma expressão, podemos utilizar a função `dump()` ou a função `Meta.show_sexpr`.

In [17]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol /
        2: Int64 2
        3: Int64 2


In [18]:
Meta.show_sexpr(ex)

(:call, :+, 1, (:call, :/, 2, 2))

Usando essas funções, fica claro, por exemplo, que nosso código primeiro avalia a divisão (`2 / 2`), para depois realizar a soma. 
Assim, esses comando podem ser úteis caso você tenha dúvida de como Julia está processando uma linha de código específica.

## Outro Exemplo
Imagine que um colega seu possui um monte de scripts em Julia. O que você gostaria de fazer é rodar o código dele,
e sempre que uma variável do tipo `Importante` for gerada, você quer salvar o nome e valor dessa variável numa pasta.
Com metaprogramação, podemos passar o script do nosso como uma expressão, e executa cada comando individualmente,
tentando se o tipo da variável é o correto.

In [21]:
codigo = """
#### Código do seu amigo
struct Importante valor::Real end
y = 0.4
x = 1 * y
x = Importante(x)
x.valor + y
z = Importante(x.valor + y)
store = z.valor + y + x.valor
"""

module Amigo

export x
x = 1
end

store = []
for code in split(codigo,"\n")
    ex = Meta.parse(code)
    if ex !== nothing
        Core.eval(Amigo, ex)
        if String(ex.head) == "="
            if typeof(Core.eval(Amigo,ex.args[1])) <: Amigo.Importante
                push!(store,(ex.args[1],Core.eval(Amigo, ex.args[1]).valor))
            end
        end
    end
end

store

2-element Vector{Any}:
 (:x, 0.4)
 (:z, 0.8)

O código funciona! Mas você deve estar se perguntando o que esse `module Amigo` está fazendo? O seu proprósito é isolar 
o código que está rodando do seu amigo, com o código que estamos rodando. Por exemplo, o código do seu amigo
termina com a linha `store = z.valor + y + x.valor`. Ou seja, ele está definindo uma variável `store`, que tem o mesmo
nome da variável que estamos usando para rodar o nosso código. Se usássemos o `Core.eval(ex)`, sem o `Amigo`, isso
acabaria alterando o valor da nossa variável `store`, causando problemas.

Você pode ainda estar se perguntando, *Mas afinal isso tem realmente algum uso prático?!* E a reposta é **sim**.
Inclusive, o tipo de problema que resolvemos no exmemplo acima é baseado num problema real que tive que
resolver quando programando o pacote `NotebookToLaTeX.jl`. Esse pacote converte notebooks Jupyter e Pluto para arquivos em LaTeX.
Um dos desafios com notebooks Pluto é que eles são nada mais que scripts em Julia, isto é, arquivos `.jl`.
Diferente do Jupyter, o Pluto não guarda os plots gerados dentro dele, sendo assim necessário rodar o código para gerar as figuras.

Meu desafio então era descobrir em que momento um plot estava sendo gerado, para poder então salvar a figura em uma pasta para
que o arquivo LaTeX pudesse ler.
Similar ao que tivemos no exemplo acima, meu código rodava cada expressão do notebook e checava para ver se era do tipo adequado.
Foi inclusive resolvendo esse problema que a comunidade de Julia sugeriu o uso de `module` como maneira de evitar
variáveis trocando valores em lugares indesejados.

## Macros
Essa seção se baseia [nesse excelente tutorial](https://www.youtube.com/watch?v=JePBb9-ychE&t=130s).

Macros são o equivalente de funções em metaprogramação. Em Julia, usa-se `@` para indicar que algo é uma função.
Veja o exemplo abaixo da macro `@show`.

In [82]:
x = 10
y = 5

@show y + x;

y + x = 15


A macro está incrementando o nosso código com mais código, e rodando. Podemos usar a macro `@macroexpand` para
mostrar exatamente o que está sendo executado quando rodamos `@show y + x;`.

In [83]:
@macroexpand @show y + x

quote
    Base.println("y + x = ", Base.repr(begin
                #= show.jl:1047 =#
                local var"#216#value" = y + x
            end))
    var"#216#value"
end

Veja que a expressão "quote" simboliza que essa expressão é código em Julia. Ou seja, poderíamos copiar e 
colar o que está dentro desse "quote" no nosso REPL. A segunda expressão estranha é esse comentário
"#= show.jl:955 =#". Neste caso, esse comentário está informando em que linha do source code está definida essa função.
Por fim, temos `var"#788#value"`. Isso é o nome da variável que o código criou. Esse nome é estranho
justamente com o propósito de evitar que já tenha essa variável no se código.

Vamos agora definir nossa primeira macro.

In [84]:
### IMPORTANTE - Essa macro está incorreta! Explicaremos abaixo.
macro inception(f, ex)
    return quote
        x = $ex
        y = $f(x)
        z = $f(y)
        x, y, z
    end
end

@inception sin 1

(1, 0.8414709848078965, 0.7456241416655579)

Vamos explicar o que está acontecendo. A macro recebe dois argumentos, e retorna uma
expressão em Julia, que está dentro do bloco `quote`. A macro irá rodar essa expressão ao final.
O `$ex` e `$f` agem como interpoladores em strings, ou seja, o usuário passa `f` e interpolamos
o `f` do usuário dentro da nossa expressão. Por fim, note que nosso bloco `quote` não tem o `return`.
Isso é deliberado e necessário, pois queremos que ele retorne essas variáveis e não a expressão `return x, y, z`.
Se você colocar o `return`, você não conseguirá usar os valores que saem dessa macro (sugiro testar caso não tenha entendido).

A macro acima parece funcionar, mas na verdade ela é defeituosa.
Qual o defeito? Veja,

In [85]:
x = 1

@inception sin x

LoadError: UndefVarError: #220#x not defined

Esse erro é estranho. Ele diz que a variável `##220#x`não está definida. Mas de fato, não definimos isso em canto nenhum.
Vamos ver o que está acontecendo por debaixo dos panos.

In [87]:
@macroexpand(@inception sin 1)

quote
    #= In[84]:4 =#
    var"#226#x" = 1
    #= In[84]:5 =#
    var"#227#y" = Main.sin(var"#226#x")
    #= In[84]:6 =#
    var"#228#z" = Main.sin(var"#227#y")
    #= In[84]:7 =#
    (var"#226#x", var"#227#y", var"#228#z")
end

Nossa macro está criando essas variáveis com nomes estranhos, e.g. `var"#68#x`. O que está havendo? 
Isso na verdade é um artifício que Julia faz para nos ajudar. Através da função `gensym()`, Julia cria
essas variáveis com nomes estranhos para evitar que essas variáveis criadas na macro se confundam com
as variáveis que já temos no nosso código.

Para ver porque isso é necessário, vamos usar a função `esc()`.
Essa função força a macro a não usar as variáveis `gensym`.

In [88]:
macro inception(f, ex)
    return esc(quote
        x = $ex
        y = $f(x)
        z = $f(y)
        x, y, z
    end)
end
z = 10
x = 1
@inception sin x

(1, 0.8414709848078965, 0.7456241416655579)

Parce funcionar, porém... 

In [89]:
z

0.7456241416655579

Nossa macro alterou o valor da variável `z`!  Esse comportamento pode não ser desejável, pois perderíamos o controle de quais
variáveis estão mudando quando rodamos uma macro, principalmente macros definidas em outros pacotes.
Como resolver esses problemas? Ou seja, como rodar `@inception sin x` sem alterar os valores
de variáveis fora da macro?

Não existe uma única resposta, mas eis uma solução elegante.

In [93]:
macro inception(f, ex)
    f = esc(f)
    ex = esc(ex)
    return quote
        x = $ex
        y = $f(x)
        z = $f(y)
        x, y, z
    end
end
z = 10
x = 1
r = @inception sin x

r,z

((1, 0.8414709848078965, 0.7456241416655579), 10)

### Manipulando Expressões

Já vimos como criar exp

In [ ]:
code = Meta.parse("j = i^2")

In [ ]:
typeof(code)

In [ ]:
dump(code)

In [ ]:
code.args

In [ ]:
code.head

In [ ]:
dump(code.args[2])

In [ ]:
code2 = copy(code)
code2.args[1] = :k
code2.args[2].args[3] = 3
code2

Em Julia, temos o tipo `Symbol` que pode ser entendido como uma expressão antes de ser
avaliada. Por exemplo, caso escrevamos `x = 2`. Ao escrever `x`, o compilador
irá avaliar essa expressão, retornando `2`. Entretanto, as vezes queremos
nos referir não ao que está dentro de `x`, mas ao símbolo em si. Para isso,
usamos `:x`.

In [ ]:
Symbol("x") == :x

Da mesma forma, podemos falar sobre a expressão `x + y` usando `:(x + y)`. Considere a seguinte situação,
você está escrevendo uma série de comandos no REPL. Você percebe que declarou tantas variáveis que não sabe mais
qual delas está armazenando o valor `10`. Julia permite que você faça isso de dentro da própria linguagem.

In [ ]:
x = 1
y = 10
z = 2

for i in 'x':'z'
    v = Meta.parse("$i")
    if eval(v) == 10
        return v
    end
end

A função `eval` está justamente avaliando a expressão dentro de `v`. O que fizemos no código acima foi iterar pela
nossas variáveis nos referindo a elas pelo seu símbolo. Imagine agora que cada uma das letras do alfabeto
foi definida como uma variável e queremos descobrir novamente qual delas armazena `10`. Como faríamos
isso sem metaprogramação? Possívelmente teríamos que manualmente escrever algo do tipo
`for v in [a,b,c,...,z]`. Com metaprogramação, podemos simplesmente escrever o loop usando `'a':'z'`.

### Referências
* [Documentação de Julia sobre Metaprogramação](https://docs.julialang.org/en/v1/manual/metaprogramming/);
* [Explicação sobre Símbolos em Julia](https://stackoverflow.com/questions/23480722/what-is-a-symbol-in-julia#23482257);